In [ ]:
import lib._util.visualplot as vp
import lib._util.fileproc as fp
import lib._util.dimreduce as dr

In [ ]:
import pandas as pd
import numpy as np
import copy
import glob

# Plotly
import plotly.express as px
import plotly.graph_objects as go

# Time measurement
import time
from datetime import timedelta, datetime

# Sound notification
import winsound

# Useful Functions

In [ ]:
SOURCE_PATH_DATA = 'resources/data/'
OUT_PATH_GRAPH   = 'resources/output/eda/graph/'
OUT_PATH_FILE    = 'resources/output/eda/file/'

def time_taken(seconds):
    print(f'\nTime Taken: {str(timedelta(seconds=seconds))}')
    winsound.Beep(frequency=1000, duration=100)
    winsound.Beep(frequency=1500, duration=50)

# Phase 1 - Data Loading

In [ ]:
def load_data(filename):
    source_file = f'{SOURCE_PATH_DATA}{filename}'
    df_chunks   = pd.read_csv(source_file, sep=';', dtype={'number': str},
                              parse_dates=['draw_date'],
                              date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),
                              chunksize=50_000)
    df = pd.concat(df_chunks)
    
    # Separate by company
    df_dict = dict()
    for company in df['company_code'].unique():
        df_dict[company] = df[df['company_code'] == company].copy()
    
    return df_dict

In [ ]:
EXEC_START = time.time()

df_dict = load_data('4D_result_1985-04-25_2019-12-31.csv')
print(df_dict.keys())

EXEC_END = time.time()
time_taken(EXEC_END - EXEC_START)

In [ ]:
def faststat(df_dict):
    for key, df in df_dict.items():
        print(key)
        vp.faststat(df)
        print()

In [ ]:
faststat(df_dict)

In [ ]:
def histogram(df, subfolder, subtitle='Histogram', layout_height=None):
    vp.histogram(df, title=f'{subfolder} - {subtitle}', out_path=f'{OUT_PATH_GRAPH}{subfolder}/', layout_height=layout_height)

def generate_histogram(df_dict, columns=None, subtitle='Histogram', layout_height=None):
    for key, df in df_dict.items():
        tmp_df = df.copy()
        if columns is not None:
            tmp_df = df[columns].copy()
            
        histogram(tmp_df, subfolder=key, subtitle=f'{key} - {subtitle}', layout_height=layout_height)

In [ ]:
generate_histogram(df_dict, subtitle='Phase 1 - Histogram')

# Phase 2 - Data Preparation

- Drop non-informative field
- Handle invalid position

In [ ]:
for key, df in df_dict.items():
    # Drop company column
    df.drop(columns=['company_code'], inplace=True)
    
    # Remove invalid number
    df = df[df['number'] != '----'].copy()
    
    df_dict[key] = df

In [ ]:
# Handle invalid position on DMC due to duplication
tmp_df = df_dict['DMC'].copy()
tmp_df = tmp_df[tmp_df['position'] <= 10]
df_dict['DMC'] = tmp_df.copy()

del tmp_df

In [ ]:
generate_histogram(df_dict, subtitle='Phase 2 - Histogram')

# Phase 3 - Feature Engineering

- Populate number features

In [ ]:
def number_feature(df_dict):
    # Load feature data
    source_file = f'{SOURCE_PATH_DATA}number_category.csv'
    feature_df  = pd.read_csv(source_file, sep=';', dtype={'number': str})
    
    newdf_dict = copy.deepcopy(df_dict)
    for key, df in newdf_dict.items():
        df = df.merge(feature_df, on='number', how='left')
        newdf_dict[key] = df
    
    return newdf_dict

def price_feature(df_dict):
    # Reference: https://www.magnum4d.my/en/4d-game
    price_dict = {
        'FST': 2500,
        'SCD': 1000,
        'TRD': 500,
        'SP': 180,
        'CONS': 60
    }
    
    newdf_dict = copy.deepcopy(df_dict)
    for key, df in newdf_dict.items():
        df['price'] = df['category'].map(price_dict)
    return newdf_dict

In [ ]:
df_dict = number_feature(df_dict)
df_dict = price_feature(df_dict)

In [ ]:
faststat(df_dict)

In [ ]:
generate_histogram(df_dict, subtitle='Phase 3 - Histogram', layout_height=1024)

# Phase 4 - RFM Analysis

- Calculate recency, frequency and monetary values for lottery number
- Assign RFM quantiles
- Assign RFM segments
- Calculate RFM scores
- Assign RFM levels

In [ ]:
def generate_rfm(df_dict, groupby, n_group=4):
    newdf_dict = copy.deepcopy(df_dict)
    for key, df in newdf_dict.items():
        # Calculate RFM values
        df = rfm_value(df, groupby)
        
        # Calculate average monetary
        df['monetary'] = df['monetary'] / df['frequency']
        df['monetary'] = np.round(df['monetary'], 2)
        
        # Assign RFM quantiles
        df = rfm_quantitle(df, 'recency', n_group=n_group, inverse=True)
        df = rfm_quantitle(df, 'frequency', n_group=n_group)
        df = rfm_quantitle(df, 'monetary', n_group=n_group)
        
        # Assign RFM segments
        df = rfm_segment(df)
        
        # Calculate RFM scores
        df = rfm_score(df)
        
        # Assign RFM levels
        # df = rfm_level(df)
        
        newdf_dict[key] = df
    
    return newdf_dict

def rfm_value(df, groupby, target_date=None):
    if target_date is None:
        target_date = pd.Timestamp(datetime.now().strftime('%Y-%m-%d'))

    rfm_df = df.groupby(groupby).agg(
        recency=('draw_date', lambda x: (target_date - x.max()).days),
        frequency=('category', 'count'),
        monetary=('price', 'sum')
    ).reset_index()

    return rfm_df

def rfm_quantitle(df, column, n_group=4, inverse=False):
    labels  = [x for x in range(n_group, 0, -1)] if inverse else [x for x in range(1, n_group +1)]
    uniques = df[column].unique()

    if len(uniques) == 1:
        quantile_dict = {x: 1 for x in uniques}
    else:
        quantile_dict = dict(zip(uniques, pd.qcut(uniques, q=n_group, labels=labels)))
    df[f'{column}_quantile'] = df[column].map(quantile_dict)

    return df

def rfm_segment(df):
    df['rfm_segment'] = df['recency_quantile'].astype(str) + '_' + df['frequency_quantile'].astype(str) + '_' + df['monetary_quantile'].astype(str)
    return df

def rfm_score(df):
    df['rfm_score'] = df[['recency_quantile', 'frequency_quantile', 'monetary_quantile']].sum(axis=1)
    return df

def rfm_level(df):
    # Reference: https://towardsdatascience.com/recency-frequency-monetary-model-with-python-and-how-sephora-uses-it-to-optimize-their-google-d6a0707c5f17
    df.loc[df['rfm_score'] <= 3, 'rfm_level'] = 'Activation Required (6)'

    df.loc[(df['rfm_score'] <= 5) &
           (df['rfm_level'].isna() == True), 'rfm_level'] = 'Needs Attention (5)'

    df.loc[(df['rfm_score'] <= 7) &
           (df['rfm_level'].isna() == True), 'rfm_level'] = 'Promising (4)'

    df.loc[(df['rfm_score'] <= 9) &
           (df['rfm_level'].isna() == True), 'rfm_level'] = 'Potential (3)'

    df.loc[(df['rfm_score'] <= 11) &
           (df['rfm_level'].isna() == True), 'rfm_level'] = 'Champions (2)'

    df.loc[(df['rfm_score'] <= 12) &
           (df['rfm_level'].isna() == True), 'rfm_level'] = 'Best of the Best (1)'

    # Reference: https://www.optimove.com/resources/learning-center/rfm-segmentation
    # RFM score: 8 - 9
    df.loc[df['rfm_segment'].isin(['4_1_4', '4_1_3']), 'rfm_level'] = 'High Profit New Number (3)'
    # RFM score: 7 - 9
    df.loc[df['rfm_segment'].isin(['3_4_1', '3_3_1', '4_4_1', '4_3_1']), 'rfm_level'] = 'Lowest Profit Active Number (3.5)'
    # RFM score: 7 - 9
    df.loc[df['rfm_segment'].isin(['1_3_3', '1_3_4', '1_4_3', '1_4_4']), 'rfm_level'] = 'Best Number At Risk (3.5)'

    return df

### Number RFM

In [ ]:
rfmdf_dict = generate_rfm(df_dict, groupby='number', n_group=10)

In [ ]:
generate_histogram(rfmdf_dict, subtitle='Phase 4 - Histogram', layout_height=1024)

In [ ]:
def boxplot(df, x_col, y_cols, subfolder, subtitle='Box-Plot', layout_height=None):
    vp.boxplot(df[y_cols + [x_col]], x_col=x_col,
               title=f'{subfolder} - {subtitle}', out_path=f'{OUT_PATH_GRAPH}{subfolder}/', layout_height=layout_height)
    
def generate_boxplot(df_dict, x_col, y_cols, subtitle='Box-Plot', layout_height=None):
    for key, df in df_dict.items():
        boxplot(df, x_col, y_cols, subfolder=key, subtitle=subtitle, layout_height=layout_height)

In [ ]:
generate_boxplot(rfmdf_dict, x_col='recency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Recency Quantile')

generate_boxplot(rfmdf_dict, x_col='frequency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Frequency Quantile')

generate_boxplot(rfmdf_dict, x_col='monetary_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Monetary Quantile')

In [ ]:
for key, df in rfmdf_dict.items():
    df['left_digits']  = df['number'].str.slice(stop=2)
    df['right_digits'] = df['number'].str.slice(start=2, stop=4)

In [ ]:
def number_heatmap(df, z_col, subfolder, subtitle='Heatmap', layout_height=None):
    vp.heatmap(df, x_col='left_digits', y_col='right_digits', z_col=z_col,
               colorscale='Viridis',
               text='Number: ' + df['number'] + '<br>RFM Segment: ' + df['rfm_segment'],
               hoverinfo='text',
               title=f'{subfolder} - {subtitle}', out_path=f'{OUT_PATH_GRAPH}{subfolder}/', layout_height=layout_height)

def generate_number_heatmap(df_dict, z_col, subtitle='Heatmap', layout_height=None):
    for key, df in df_dict.items():
        number_heatmap(df, z_col, subfolder=key, subtitle=subtitle, layout_height=layout_height)

In [ ]:
generate_number_heatmap(rfmdf_dict, z_col='rfm_score',
                        subtitle='Phase 4 - Heatmap - RFM Score')

generate_number_heatmap(rfmdf_dict, z_col='recency_quantile',
                        subtitle='Phase 4 - Heatmap - Recency Quantile')

generate_number_heatmap(rfmdf_dict, z_col='frequency_quantile',
                        subtitle='Phase 4 - Heatmap - Frequency Quantile')

generate_number_heatmap(rfmdf_dict, z_col='monetary_quantile',
                        subtitle='Phase 4 - Heatmap - Monetary Quantile')

### Pattern RFM

In [ ]:
rfmdf_dict = generate_rfm(df_dict, groupby='pattern', n_group=4)

In [ ]:
generate_boxplot(rfmdf_dict, x_col='recency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Recency Quantile - Pattern')

generate_boxplot(rfmdf_dict, x_col='frequency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Frequency Quantile - Pattern')

generate_boxplot(rfmdf_dict, x_col='monetary_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Monetary Quantile - Pattern')

In [ ]:
def bar(df, x_col, y_cols, subfolder, subtitle='Bar', layout_height=None):
    vp.bar(df, x_col=x_col, y_cols=y_cols,
           title=f'{subfolder} - {subtitle}', out_path=f'{OUT_PATH_GRAPH}{subfolder}/', layout_height=layout_height)
    
def generate_bar(df_dict, x_col, y_cols, subtitle='Bar', layout_height=None):
    for key, df in df_dict.items():
        bar(df, x_col, y_cols, subfolder=key, subtitle=subtitle, layout_height=layout_height)

In [ ]:
y_cols = ['recency_quantile', 'recency',
          'frequency_quantile', 'frequency',
          'monetary_quantile', 'monetary',
          'rfm_score']
generate_bar(rfmdf_dict, x_col='pattern', y_cols=y_cols,
             subtitle='Phase 4 - Bar - Pattern RFM', layout_height=1024)

### Group RFM

In [ ]:
for groupby in ['group_4', 'group_3', 'group_2', 'group_1']:
    rfmdf_dict = generate_rfm(df_dict, groupby=groupby, n_group=4)
    
    generate_boxplot(rfmdf_dict, x_col='recency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                     subtitle=f'Phase 4 - Box-Plot - Recency Quantile - {groupby.title()}')
    
    generate_boxplot(rfmdf_dict, x_col='frequency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                     subtitle=f'Phase 4 - Box-Plot - Frequency Quantile - {groupby.title()}')
    
    generate_boxplot(rfmdf_dict, x_col='monetary_quantile', y_cols=['recency', 'frequency', 'monetary'],
                     subtitle=f'Phase 4 - Box-Plot - Monetary Quantile - {groupby.title()}')
    
    y_cols = ['recency_quantile', 'recency',
              'frequency_quantile', 'frequency',
              'monetary_quantile', 'monetary',
              'rfm_score']
    generate_bar(rfmdf_dict, x_col=groupby, y_cols=y_cols,
                 subtitle=f'Phase 4 - Bar - {groupby.title()} RFM', layout_height=1024)

### Odd Even RFM

In [ ]:
rfmdf_dict = generate_rfm(df_dict, groupby='odd_even', n_group=4)

In [ ]:
generate_boxplot(rfmdf_dict, x_col='recency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Recency Quantile - Odd-Even')

generate_boxplot(rfmdf_dict, x_col='frequency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Frequency Quantile - Odd-Even')

generate_boxplot(rfmdf_dict, x_col='monetary_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Monetary Quantile - Odd-Even')

In [ ]:
y_cols = ['recency_quantile', 'recency',
          'frequency_quantile', 'frequency',
          'monetary_quantile', 'monetary',
          'rfm_score']
generate_bar(rfmdf_dict, x_col='odd_even', y_cols=y_cols,
             subtitle='Phase 4 - Bar - Odd-Even RFM', layout_height=1024)

### Big Small RFM

In [ ]:
rfmdf_dict = generate_rfm(df_dict, groupby='big_small', n_group=4)

In [ ]:
generate_boxplot(rfmdf_dict, x_col='recency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Recency Quantile - Big-Small')

generate_boxplot(rfmdf_dict, x_col='frequency_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Frequency Quantile - Big-Small')

generate_boxplot(rfmdf_dict, x_col='monetary_quantile', y_cols=['recency', 'frequency', 'monetary'],
                 subtitle='Phase 4 - Box-Plot - Monetary Quantile - Big-Small')

In [ ]:
y_cols = ['recency_quantile', 'recency',
          'frequency_quantile', 'frequency',
          'monetary_quantile', 'monetary',
          'rfm_score']
generate_bar(rfmdf_dict, x_col='big_small', y_cols=y_cols,
             subtitle='Phase 4 - Bar - Big-Small RFM', layout_height=1024)

# Phase 5 - Moving RFM Analysis

- Calculate RFM values on each periods

In [ ]:
def generate_moving_rfm(df_dict, groupby, n_group=4, subtitle=None):
    for key, df in df_dict.items():
        print(key)
        
        # Gather dates
        dates = np.sort(df['draw_date'].unique())
        dates = list(map(lambda x: pd.Timestamp(x), dates))
        
        # Gather years
        years = list(set([x.year for x in dates]))
        
        for year in years:
            # Filter for dates fall under year
            filter_dates = [x for x in dates if x.year == year]
            
            # Perform RFM analysis on each dates
            filter_list = []
            for date in filter_dates:
                # Date filtering
                filter_df = df[df['draw_date'] <= date].copy()

                # Calculate RFM values
                filter_df = rfm_value(filter_df, groupby=groupby, target_date=date)

                # Calculate average monetary
                filter_df['monetary'] = filter_df['monetary'] / filter_df['frequency']
                filter_df['monetary'] = np.round(filter_df['monetary'], 2)

                # Assign RFM quantiles
                filter_df = rfm_quantitle(filter_df, 'recency', n_group=n_group, inverse=True)
                filter_df = rfm_quantitle(filter_df, 'frequency', n_group=n_group)
                filter_df = rfm_quantitle(filter_df, 'monetary', n_group=n_group)

                # Assign RFM segments
                filter_df = rfm_segment(filter_df)

                # Calculate RFM scores
                filter_df = rfm_score(filter_df)

                # Assign RFM levels
                # filter_df = rfm_level(filter_df)

                # Collect data
                filter_df['date'] = date
                filter_list.append(filter_df.to_dict())

            # Compile data for each dates
            rfm_df = pd.concat([pd.DataFrame(x) for x in filter_list]).reset_index(drop=True)
            
            # Export moving RFM
            fp.generate_csv(rfm_df, out_path=f'{OUT_PATH_FILE}{key}', out_filename=f'{key} - Moving RFM - {year}.csv', export_index=False)

In [ ]:
EXEC_START = time.time()

generate_moving_rfm(df_dict, groupby='number', n_group=10)

EXEC_END = time.time()
time_taken(EXEC_END - EXEC_START)

# Phase 6 - Complete Set

In [ ]:
def load_moving_rfm(company_code, start_year=None, end_year=None):
    files      = glob.glob(f'{OUT_PATH_FILE}{company_code}/{company_code} - Moving RFM - *.csv')
    files_dict = {x: int(x[x.index('.csv') - 4: x.index('.csv')]) for x in files}
    files      = [k for k,v in files_dict.items()
                  if (True if start_year is None else v >= start_year) and (True if end_year is None else v <= end_year)]
    
    dfs = []
    for file in files:
        print(file)
        df_chunks   = pd.read_csv(file, sep=';', dtype={'number': str},
                                  parse_dates=['date'],
                                  date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),
                                  chunksize=50_000)
        df = pd.concat(df_chunks)
        dfs.append(df)
        
    return pd.concat(dfs)

In [ ]:
# Magnum
# - Operate from 1985-04-25, and 2013-11-17 is the date where all numbers occurs at least once
# - Last number to occur is 6962
company_code = 'MAG'
df = load_moving_rfm(company_code, start_year=2013, end_year=2013)
df = df[df['date'] <= '2013-11-17'].reset_index().copy()

# # Da Ma Cai
# # - Operate from 1990-01-06, and 2016-11-27 is the date where all numbers occurs at least once
# # - Last number to occur is 6771
# company_code = 'DMC'
# df = load_moving_rfm(company_code, start_year=2016, end_year=2016)
# df = df[df['date'] <= '2016-11-27'].reset_index().copy()

# # Sports Toto
# # - Operate from 1992-05-06, and 2015-05-27 is the date where all numbers occurs at least once
# # - Last number to occur is 5488
# company_code = 'ST'
# df = load_moving_rfm(company_code, start_year=2015, end_year=2015)
# df = df[df['date'] <= '2015-05-27'].reset_index().copy()

In [ ]:
rfm_df = df.groupby('number').agg(
    recency=('recency', 'last'),
    frequency=('frequency', 'last'),
    monetary=('monetary', 'last'),
    recency_quantile=('recency_quantile', 'last'),
    frequency_quantile=('frequency_quantile', 'last'),
    monetary_quantile=('monetary_quantile', 'last'),
    rfm_score=('rfm_score', 'last'),
    first_date=('date', 'min'),
    last_date=('date', 'max')
).reset_index()

rfm_df.shape

In [ ]:
boxplot(rfm_df, x_col='recency_quantile', y_cols=['recency', 'frequency', 'monetary'],
        subfolder=company_code,
        subtitle='Phase 5 - Box-Plot - Recency Quantile - Complete Set')

In [ ]:
boxplot(rfm_df, x_col='frequency_quantile', y_cols=['recency', 'frequency', 'monetary'],
        subtitle='Phase 5 - Box-Plot - Frequency Quantile - Complete Set')

In [ ]:
boxplot(rfm_df, x_col='monetary_quantile', y_cols=['recency', 'frequency', 'monetary'],
        subtitle='Phase 5 - Box-Plot - Monetary Quantile - Complete Set')